# Trabalho Prático 1 - O Analista de Dados
**Disciplina:** ADS308 - Inteligência Artificial  \n
**Aluno(a):** Luiz Guilherme Gomes Coelho - 25880  \n
**Data:** 12/02/2026

Neste notebook eu faço a limpeza e uma análise exploratória de um dataset de vendas.

## 1) Carregamento e inspeção inicial

In [2]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# estilo básico dos gráficos
sns.set_style('whitegrid')

# nome do arquivo (se estiver na mesma pasta do notebook)
arquivo_csv = 'dataset_vendas_sujo_25994.csv'

# se não achar o arquivo localmente, tenta upload (Colab)
if not os.path.exists(arquivo_csv):
    from google.colab import files
    print('Arquivo não encontrado. Faça upload do CSV:')
    enviado = files.upload()
    arquivo_csv = next(iter(enviado))

# lendo o dataset
df = pd.read_csv(arquivo_csv)
print('Dataset carregado:', arquivo_csv)

Dataset carregado: dataset_vendas_sujo_25994.csv


In [3]:
print('Primeiras 5 linhas:')
display(df.head())

print('Últimas 5 linhas:')
display(df.tail())

Primeiras 5 linhas:


,ID_Pedido,Data_Venda,Categoria,Preco_Unitario,Quantidade,Idade_Cliente
0,1,2023-12-21 00:00:00,Roupas,NaN,9.0,52.0
1,2,2023-03-06 00:00:00,Beleza,159.17095149537838,6.0,60.0
2,3,2023-05-29 00:00:00,Beleza,97.56851425999506,5.0,24.0
3,4,2023-08-21 00:00:00,Casa,200.69318993603952,3.0,62.0
4,5,2023-04-23 00:00:00,Casa,166.64299089108152,2.0,35.0


Últimas 5 linhas:


,ID_Pedido,Data_Venda,Categoria,Preco_Unitario,Quantidade,Idade_Cliente
5045,5046,2023-09-20 00:00:00,Eletrônicos,187.08399695761653,2.0,NaN
5046,5047,2023-12-21 00:00:00,Beleza,200.14642344076879,2.0,54.0
5047,5048,2023-09-20 00:00:00,Beleza,142.7899853031845,6.0,57.0
5048,5049,2023-10-22 00:00:00,Roupas,196.30007978143982,4.0,20.0
5049,5050,2023-12-13 00:00:00,Casa,155.50869215335698,7.0,46.0


In [4]:
print('Resumo rápido do dataset:')
df.info()
print('\nTotal de linhas:', len(df))
print('Total de colunas:', df.shape[1])

Resumo rápido do dataset:
<class 'pandas.DataFrame'>
RangeIndex: 5050 entries, 0 to 5049
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID_Pedido       5050 non-null   int64  
 1   Data_Venda      5050 non-null   str    
 2   Categoria       5050 non-null   str    
 3   Preco_Unitario  4798 non-null   str    
 4   Quantidade      4949 non-null   float64
 5   Idade_Cliente   4898 non-null   float64
dtypes: float64(2), int64(1), str(3)
memory usage: 236.8 KB

Total de linhas: 5050
Total de colunas: 6


## 2) Limpeza e tratamento dos dados
Nessa parte eu vou:
- ver os dados faltantes;
- tratar outliers usando Z-Score;
- transformar a coluna de data em `datetime`.

In [5]:
print('Valores nulos por coluna:')
nulos = df.isnull().sum()
display(nulos[nulos > 0])

Valores nulos por coluna:


Preco_Unitario    252
Quantidade        101
Idade_Cliente     152
dtype: int64

In [6]:
# cópia para não mexer no dataset original
df_limpo = df.copy()

# separando colunas numéricas e de texto
cols_num = df_limpo.select_dtypes(include='number').columns
cols_txt = df_limpo.select_dtypes(exclude='number').columns

# preenchendo nulos das numéricas com mediana
df_limpo[cols_num] = df_limpo[cols_num].fillna(df_limpo[cols_num].median())

# preenchendo nulos das de texto com moda
for coluna in cols_txt:
    if df_limpo[coluna].isnull().sum() > 0:
        valor_moda = df_limpo[coluna].mode()
        if len(valor_moda) > 0:
            df_limpo[coluna] = df_limpo[coluna].fillna(valor_moda[0])
        else:
            df_limpo[coluna] = df_limpo[coluna].fillna('Não informado')

print('Nulos depois do tratamento:')
display(df_limpo.isnull().sum())

Nulos depois do tratamento:


ID_Pedido         0
Data_Venda        0
Categoria         0
Preco_Unitario    0
Quantidade        0
Idade_Cliente     0
dtype: int64

In [7]:
# tentando achar automaticamente uma coluna de data
possiveis_datas = [c for c in df_limpo.columns if 'data' in c.lower() or 'date' in c.lower()]

if len(possiveis_datas) > 0:
    coluna_data = possiveis_datas[0]
    df_limpo[coluna_data] = pd.to_datetime(df_limpo[coluna_data], errors='coerce', dayfirst=True)
    print('Coluna de data convertida:', coluna_data)
else:
    coluna_data = None
    print('Não encontrei coluna de data automaticamente.')

Coluna de data convertida: Data_Venda


C:\Users\Usuario\AppData\Local\Temp\ipykernel_13168\3152344709.py:6: UserWarning: Parsing dates in %Y-%m-%d %H:%M:%S format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  df_limpo[coluna_data] = pd.to_datetime(df_limpo[coluna_data], errors='coerce', dayfirst=True)


In [8]:
# tratando outliers com regra de 3 desvios (Z-Score)
cols_num = df_limpo.select_dtypes(include='number').columns
print('Colunas numéricas:', list(cols_num))

for coluna in cols_num:
    media = df_limpo[coluna].mean()
    desvio = df_limpo[coluna].std()

    if desvio == 0 or pd.isna(desvio):
        continue

    z = (df_limpo[coluna] - media) / desvio
    eh_outlier = z.abs() > 3

    if eh_outlier.sum() > 0:
        minimo = media - 3 * desvio
        maximo = media + 3 * desvio
        df_limpo.loc[df_limpo[coluna] < minimo, coluna] = minimo
        df_limpo.loc[df_limpo[coluna] > maximo, coluna] = maximo
        print(f'{coluna}: {int(eh_outlier.sum())} outliers tratados')

Colunas numéricas: ['ID_Pedido', 'Quantidade', 'Idade_Cliente']
Idade_Cliente: 20 outliers tratados


## 3) Análise Exploratória (EDA)

In [9]:
# escolhendo colunas automaticamente pelo nome
col_venda = next((c for c in df_limpo.columns if 'venda' in c.lower() or 'valor' in c.lower() or 'faturamento' in c.lower() or 'total' in c.lower()), None)
col_qtd = next((c for c in df_limpo.columns if 'quantidade' in c.lower() or 'qtd' in c.lower() or 'qtde' in c.lower()), None)
col_preco = next((c for c in df_limpo.columns if 'preco' in c.lower() or 'preço' in c.lower()), None)
col_categoria = next((c for c in df_limpo.columns if 'categoria' in c.lower() or 'produto' in c.lower()), None)

print('Coluna de venda:', col_venda)
print('Coluna de quantidade:', col_qtd)
print('Coluna de preço:', col_preco)
print('Coluna de categoria:', col_categoria)
print('Coluna de data:', coluna_data)

print('\nSe alguma ficar como None, é só trocar manualmente aqui nesta célula.')

Coluna de venda: Data_Venda
Coluna de quantidade: Quantidade
Coluna de preço: Preco_Unitario
Coluna de categoria: Categoria
Coluna de data: Data_Venda

Se alguma ficar como None, é só trocar manualmente aqui nesta célula.


In [10]:
# Pergunta 1: Qual a distribuição das vendas ao longo dos meses?
if coluna_data is not None and col_venda is not None:
    base_mes = df_limpo.dropna(subset=[coluna_data]).copy()
    base_mes['mes'] = base_mes[coluna_data].dt.to_period('M').astype(str)
    vendas_mes = base_mes.groupby('mes')[col_venda].sum().reset_index()

    plt.figure(figsize=(10,4))
    plt.plot(vendas_mes['mes'], vendas_mes[col_venda], marker='o')
    plt.title('Distribuição das vendas ao longo dos meses')
    plt.xlabel('Mês')
    plt.ylabel('Total de vendas')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print('Não foi possível gerar o gráfico de vendas por mês (faltou coluna de data e/ou venda).')

TypeError: datetime64 type does not support operation 'sum'

In [ ]:
# Pergunta 2: Existe correlação entre preço e quantidade vendida?
if col_preco is not None and col_qtd is not None:
    plt.figure(figsize=(7,5))
    sns.scatterplot(data=df_limpo, x=col_preco, y=col_qtd, alpha=0.7)
    plt.title('Preço x Quantidade vendida')
    plt.xlabel('Preço')
    plt.ylabel('Quantidade')
    plt.tight_layout()
    plt.show()

    corr = df_limpo[[col_preco, col_qtd]].corr().iloc[0,1]
    print('Correlação (Pearson):', round(corr, 3))
else:
    print('Não foi possível gerar o scatter de preço x quantidade.')

In [ ]:
# Pergunta 3: Como as vendas se dividem por categoria?
if col_categoria is not None and col_venda is not None:
    vendas_categoria = df_limpo.groupby(col_categoria)[col_venda].sum().sort_values(ascending=False)

    plt.figure(figsize=(8,5))
    vendas_categoria.plot(kind='bar')
    plt.title('Vendas por categoria')
    plt.xlabel('Categoria')
    plt.ylabel('Total de vendas')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()
else:
    print('Não foi possível gerar o gráfico de vendas por categoria.')

## 4) Conclusões e insights
Preencha com base no que aparecer na execução:

1. Os meses com maior volume de vendas foram: **(preencher)**.
2. A relação entre preço e quantidade vendida foi **(fraca/moderada/forte)**, com correlação de **(preencher)**.
3. As categorias que mais contribuíram para o faturamento foram **(preencher)**.
4. O tratamento de nulos e outliers ajudou a deixar os dados mais consistentes.
5. Sugestão para a gerência: focar campanhas e estoque nas categorias e períodos de maior retorno.